In [1]:
import docx
import re
import pandas as pd

import os
os.chdir("../")


In [11]:
file_path = "data/evaluation_sets/Robinson Q&A.docx"  # Replace with your actual file path


In [20]:
import docx
import re
import pandas as pd

def extract_qa_pairs_to_df(file_path):
    """Extracts question-answer pairs from a docx file and saves them to a DataFrame.

    Args:
        file_path (str): The path to the docx file.

    Returns:
        pd.DataFrame: A DataFrame with columns "Question_Number", "Question", and "Answer".
    """

    try:
        doc = docx.Document(file_path)
        questions = []
        answers = []
        question_numbers = []
        current_question = None

        for para in doc.paragraphs:
            # Match questions (e.g., Q1:)
            question_match = re.match(r"(Q\d+[a-z]?):\s*(.*)", para.text)  # Capture question text
            if question_match:
              
                current_question = question_match.group(1)
                
                # Add the question number and text together in one text
                # current_question = question_match.group(1) + ": " + question_match.group(2)  # Combine question number and text

                question_numbers.append(current_question)  # Store question number
                questions.append(question_match.group(2))  # Store question text
                answers.append("")  # Initialize answer
            elif current_question:  # If we're within a question-answer block
                # Remove "A1:" prefix if present
                answer_text = re.sub(r"^A\d+[a-z]?:\s*", "", para.text)  
                answers[-1] += answer_text + "\n"  # Append to the last answer

        # Create DataFrame
        df = pd.DataFrame({"Question": questions, "Answer": answers})
        
        ### Inlude the question numbers to the dataframe
        # df = pd.DataFrame({"Question_Number": question_numbers, "Question": questions, "Answer": answers})
        return df

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
        return None
    except docx.opc.exceptions.PackageNotFoundError:
        print(f"Error: Invalid docx file at '{file_path}'")
        return None

# Example usage
# file_path = "Raptor Q&A2.docx"  # Replace with your actual file path
df = extract_qa_pairs_to_df(file_path)

df.head()

,Question,Answer
0,Who are the parties to the Agreement and what ...,Cloud Investments Ltd. (“Company”) and Jack Ro...
1,What is the termination notice?,According to section 4:14 days for convenience...
2,What are the payments to the Advisor under the...,According to section 6: 1. Fees of $9 per hour...
3,Can the Agreement or any of its obligations be...,1. Under section 1.1 the Advisor can’t assign ...
4,Who owns the IP?,According to section 4 of the Undertaking (App...


In [23]:
# rename the columns for consistency, change answer to ground_truth
df.columns = ['question', 'ground_truths']


In [24]:
df.head()

,question,ground_truths
0,Who are the parties to the Agreement and what ...,Cloud Investments Ltd. (“Company”) and Jack Ro...
1,What is the termination notice?,According to section 4:14 days for convenience...
2,What are the payments to the Advisor under the...,According to section 6: 1. Fees of $9 per hour...
3,Can the Agreement or any of its obligations be...,1. Under section 1.1 the Advisor can’t assign ...
4,Who owns the IP?,According to section 4 of the Undertaking (App...


In [25]:
from ragas.integrations.langsmith import upload_dataset

dataset_name = "Mere Christianity"
dataset_desc = "Synthetic testset data for Mere Christianity."

dataset = upload_dataset(testset, dataset_name, dataset_desc)

/home/hilla/code/10Academy-training/week11/contract-qa-high-precision-rag/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


NameError: name 'qa' is not defined